In [2]:
from collections import deque
from tree_sitter import Language, Parser
import os
from formast.__main__ import JAVA_LANGUAGE

parser = Parser()
parser.set_language(JAVA_LANGUAGE)

with open("..\java_files\simple.java", "rb") as f:
    code = f.read()
tree = parser.parse(code)
tree

In [3]:
from tree_sitter import Node

def print_node(node: Node, indent: str = ""):
    print(f"{indent}{node.type} [{node.start_point[0]}, {node.start_point[1]}] - [{node.end_point[0]}, {node.end_point[1]}]")
    for child in node.children:
        print_node(child, indent + "  ")

print_node(tree.root_node)

program [0, 0] - [0, 16]
  class_declaration [0, 0] - [0, 16]
    modifiers [0, 0] - [0, 6]
      public [0, 0] - [0, 6]
    class [0, 7] - [0, 12]
    identifier [0, 13] - [0, 14]
    class_body [0, 14] - [0, 16]
      { [0, 14] - [0, 15]
      } [0, 15] - [0, 16]


In [4]:
def process_tree(tree, code):
    lines = []
    def process_node(node):
        if node.type == 'identifier':
            #it was decode before
            lines.append(f'N {code[node.start_byte:node.end_byte].decode("utf-8")}')
            lines.append(f'B identifier {len(lines) - 1}')
            return len(lines) - 1
        elif node.type == 'class_body':
            children = []
            for child in node.children:
                if child.type not in ['{', '}']:
                    child_index = process_node(child)
                    if child_index is not None:
                        children.append(child_index)
            lines.append(f'B {node.type} {" ".join(map(str, children))}')
            return len(lines) - 1
        else:
            children = []
            for child in node.children:
                child_index = process_node(child)
                if child_index is not None:
                    children.append(child_index)
            lines.append(f'B {node.type} {" ".join(map(str, children))}')
            return len(lines) - 1
    process_node(tree.root_node)
    return '\n'.join(lines)

print(process_tree(tree,code))

B public 
B modifiers 0
B class 
N A
B identifier 3
B class_body 
B class_declaration 1 2 4 5
B program 6


In [5]:
def process_tree_ast(tree):
    lines = []

    def process_node(node):
        line = ""

        if node.children:
            children = []
            for child in node.children:
                child_result = process_node(child)
                if child_result is not None:
                    children.append(child_result)
            line = 'B {} {}'.format(node.type, ' '.join(map(str, children)))
        else:
            text = node.text.decode('utf-8')
            line = 'L {}'.format(text)

        lines.append('{}'.format(line))

        return len(lines) - 1 

    process_node(tree.root_node)
    return '\n'.join(lines) 

print(process_tree_ast(tree))

L public
B modifiers 0
L class
L A
L {
L }
B class_body 4 5
B class_declaration 1 2 3 6
B program 7


In [7]:
def process_tree_ast_rel(tree):
    if tree is None:
        raise ValueError("The tree object must not be None")

    lines = []

    def process_node(node):
        if node is None:
            raise ValueError("The tree object does not have the expected structure")

        line = ""

        if node.children:
            children = []
            for child in node.children:
                child_result = process_node(child)
                if child_result is not None:
                    children.append(child_result)
            line = 'B {} {}'.format(node.type, ' '.join(map(lambda x: str(x - len(lines)), children)))
        else:
            try:
                text = node.text.decode('utf-8')
            except UnicodeDecodeError:
                raise ValueError("The text of the leaf nodes must be encoded using utf-8")
            line = 'L {}'.format(text)

        lines.append('{}'.format(line))

        return len(lines) - 1

    process_node(tree.root_node)
    return '\n'.join(lines)

print(process_tree_ast_rel(tree))

L public
B modifiers -1
L class
L A
L {
L }
B class_body -2 -1
B class_declaration -6 -5 -4 -1
B program -1


In [16]:
import base64
import hashlib

def process_tree_comp(tree):
    lookup = {}
    lines = []

    def process_node(node):
        line = ""

        if node.children:
            children = []
            for child in node.children:
                child_result = process_node(child)
                if child_result is not None:
                    children.append(child_result)
            line = 'B {} {}'.format(node.type, ' '.join(map(str, children)))
        else:
            text = node.text.decode('utf-8')
            line = 'L {}'.format(text)

        idx = lookup.get(line)
        if idx is not None:
            return idx

        hash_object = hashlib.sha256(line.encode('utf-8'))
        hash_value = int.from_bytes(hash_object.digest()[:8], byteorder='big', signed=True)
        idx = base64.urlsafe_b64encode(hash_value.to_bytes(8, byteorder='big', signed=True)).rstrip(b'=').decode('ascii')
        lines.append('{} {}'.format(idx, line))
        lookup[line] = idx

        return idx

    process_node(tree.root_node)
    return '\n'.join(lines)  

print(process_tree_comp(tree))

_6xEEtLT7vg L public
iLRoi73V-RI B modifiers _6xEEtLT7vg
2usUse1FW98 L class
Jf7gA03GbKE L A
f1dtMFaAKXw L {
QRb-VDvN7n0 L }
_ETJtzenYkI B class_body f1dtMFaAKXw QRb-VDvN7n0
FwqmimPqmxY B class_declaration iLRoi73V-RI 2usUse1FW98 Jf7gA03GbKE _ETJtzenYkI
M8gnXQfEJJs B program FwqmimPqmxY


In [13]:
import base64
import hashlib

def process_tree_comp_sorted(tree):
    if tree is None:
        raise ValueError("The tree object must not be None")

    lookup = {}
    lines = []

    def process_node_sorted(node):
        if node is None:
            raise ValueError("The tree object does not have the expected structure")

        line = ""

        if node.children:
            children = []
            for child in node.children:
                child_result = process_node_sorted(child)
                if child_result is not None:
                    children.append(child_result)
            line = 'B {} {}'.format(node.type, ' '.join(map(str, children)))
        else:
            try:
                text = node.text.decode('utf-8')
            except UnicodeDecodeError:
                raise ValueError("The text of the leaf nodes must be encoded using utf-8")
            line = 'L {}'.format(text)

        idx = lookup.get(line)
        if idx is not None:
            return idx

        hash_object = hashlib.sha256(line.encode('utf-8'))
        hash_value = int.from_bytes(hash_object.digest()[:8], byteorder='big', signed=True)
        idx = base64.urlsafe_b64encode(hash_value.to_bytes(8, byteorder='big', signed=True)).rstrip(b'=').decode('ascii')
        lines.append('{} {}'.format(idx, line))
        lookup[line] = idx

        return idx

    process_node_sorted(tree.root_node)
    sorted_lines = sorted(lines, key=lambda x: x.split()[0])
    return '\n'.join(sorted_lines)

print(process_tree_comp_sorted(tree))

2usUse1FW98 L class
FwqmimPqmxY B class_declaration iLRoi73V-RI 2usUse1FW98 Jf7gA03GbKE _ETJtzenYkI
Jf7gA03GbKE L A
M8gnXQfEJJs B program FwqmimPqmxY
QRb-VDvN7n0 L }
_6xEEtLT7vg L public
_ETJtzenYkI B class_body f1dtMFaAKXw QRb-VDvN7n0
f1dtMFaAKXw L {
iLRoi73V-RI B modifiers _6xEEtLT7vg
